In [7]:
import os
import numpy as np
import pandas as pd
import keras
import shutil

In [2]:
image_path = 'celeba-dataset/img_align_celeba'
attributes_path = 'celeba-dataset/list_attr_celeba.csv'

### Load attribute data:

In [23]:
attributes_df = pd.read_csv(attributes_path)
attributes_df.columns.values

array(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair',
       'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee',
       'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male',
       'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair',
       'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young'], dtype=object)

In [40]:
attributes_to_keep = ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Brown_Hair',
       'Bushy_Eyebrows', 'Goatee',
       'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male',
       'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pointy_Nose', 'Smiling', 'Straight_Hair',
       'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Young']
attribute_data = attributes_df[attributes_to_keep]
attribute_data = attribute_data.values[:,1:] # note that index starts with 0 and image id with 1!


In [39]:
attribute_data.values

array([['000001.jpg', -1, 1, ..., 1, -1, 1],
       ['000002.jpg', -1, -1, ..., -1, -1, 1],
       ['000003.jpg', -1, -1, ..., -1, -1, 1],
       ...,
       ['202597.jpg', -1, -1, ..., -1, -1, 1],
       ['202598.jpg', -1, 1, ..., 1, -1, 1],
       ['202599.jpg', -1, 1, ..., 1, -1, 1]], dtype=object)

### Load image data:

Code below places images into correct categories

In [12]:
images = os.listdir(image_path)
image_folder = 'categories'
shutil.rmtree(tmp_folder)
for index, image in enumerate(images):
    folder_name = image.split('.')[0]
    new_folder_name = os.path.join(image_folder, folder_name)
    os.makedirs(new_folder_name)
    shutil.copyfile(os.path.join(image_path, image), os.path.join(new_folder_name, image))
print('Done!')

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000


### Define data generators

In [15]:
target_size = (128, 128)
input_shape = (target_size[0], target_size[1], 3)
batch_size = 64

data_generator = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255)

train_generator = data_generator.flow_from_directory(image_folder, 
                                                     target_size=target_size, 
                                                     batch_size=batch_size)


Found 202599 images belonging to 202599 classes.


In [50]:
x, y = train_generator.next()
y = np.argmax(y, axis = 1)

In [51]:
y.shape

(64,)

In [52]:
y

array([  4736, 171444, 180457, 116105,  18627, 116490, 196300,  22834,
       109083, 128888,  85179, 197050,  24402,  45313,  68569,   8663,
        96826, 168684, 131893,  47368, 164127,  70472,   3749,  17628,
        16920,  35498,  22065,  35125, 141796,  16631, 114906,  74834,
       140669,  36832, 171239, 150755, 112862, 101993, 101763,  18950,
       152971,  10148,  47881, 154237,  94467, 146133,  32252, 138958,
        98845,  14361,  46106, 154862, 144057,  99756,  98287,  18717,
       167885, 151531, 114854,  98252, 137688, 193766,  17365,  18892])